In [ ]:
%reset -f

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.gridspec as gridspec

from scipy.signal import butter, filtfilt
from matplotlib.patches import ConnectionPatch
from MINE.Log import Log
from MINE.Analysis import SessionAnalytics, ExperimentAnalytics
from MINE.StreamFilter import IStreamFilter, TimestampStreamFilter
from MINE.SessionFilters import ISessionFilter, ContainsStreamSessionFilter, ContainsMarkersSessionFilter
from MINE.StreamProcessing import StreamProcesses
from MINE.StreamOutput import StreamOutput

In [ ]:
def generate_filter(video_name: str):
    marker_pairs = _session_analytics.get_paired_markers("Marker", "VideoStart", "VideoEnd")

    marker_row = marker_pairs[marker_pairs["Start Marker"].str.contains(video_name, regex=False)]
    start_time = marker_row["Start Timestamp"].iloc[0]
    end_time = marker_row["End Timestamp"].iloc[0]

    return TimestampStreamFilter(start_time, end_time)

In [ ]:
def stream_filter() -> ContainsStreamSessionFilter:
    return ContainsStreamSessionFilter([
        "Marker",
        "PPG_GRN",
        "PPG_RED",
        "PPG_IR",
    ])

def marker_filter() -> ContainsMarkersSessionFilter:
    return ContainsMarkersSessionFilter("Marker", [
        "Video: Be a floater, Event: VideoStart",
        "Video: Be a floater, Event: VideoEnd",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoStart",
        "Video: RNLI Respect the water “Ladbible short film”, Event: VideoEnd",
        "Video: “Evans story”, Event: VideoStart",
        "Video: “Evans story”, Event: VideoEnd",
        "Video: “Little girl being rescued by RNLI”, Event: VideoStart",
        "Video: “Little girl being rescued by RNLI”, Event: VideoEnd",
        "Video: Alfie’s phone, Event: VideoStart",
        "Video: Alfie’s phone, Event: VideoEnd",
        "Video: “Float to Live”, Event: VideoStart",
        "Video: “Float to Live”, Event: VideoEnd",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoStart",
        "Video: Respect the Water via the NWSF\u202f‘make the right call’, Event: VideoEnd",
        "Video: “Seaside safety song”, Event: VideoStart",
        "Video: “Seaside safety song”, Event: VideoEnd",
        "Video: RNLI the breath test, Event: VideoStart",
        "Video: RNLI the breath test, Event: VideoEnd",
        "Video: RNLI Christmas bed-time story, Event: VideoStart",
        "Video: RNLI Christmas bed-time story, Event: VideoEnd",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoStart",
        "Video: “RNLI: The heart-breaking story of Liam Hall”, Event: VideoEnd",
    ])

In [ ]:
def participant_data() -> pd.DataFrame:
     return pd.DataFrame(columns=["Participant_ID", "File_Path"], data=[
         #["P01 - 535679", "V:/Data/Analysis/RNLI/participant 1.xdf"],
         #["P02 - 888462", "V:/Data/Analysis/RNLI/participant 2.xdf"],
         #["P03 - 499031", "V:/Data/Analysis/RNLI/participant 3.xdf"],
         #["P04.0 - 832362", "V:/Data/Analysis/RNLI/participant 4.0.xdf"],
         #["P04.1 - 832362", "V:/Data/Analysis/RNLI/participant 4.1.xdf"],
         ["P05", "V:/Data/Analysis/RNLI/participant 5.xdf"],
         #["P06", "V:/Data/Analysis/RNLI/participant 6.xdf"],
         #["P07", "V:/Data/Analysis/RNLI/participant 7.xdf"],
         #["P08", "V:/Data/Analysis/RNLI/participant 8.xdf"],
         #["P09", "V:/Data/Analysis/RNLI/participant 9.xdf"],
         #["P10 - 999149", "V:/Data/Analysis/RNLI/participant 10.xdf"],
         #["P11 - 153327", "V:/Data/Analysis/RNLI/participant 11.xdf"],
         #["P12", "V:/Data/Analysis/RNLI/participant 12.xdf"],
         #["P13", "V:/Data/Analysis/RNLI/participant 13.xdf"],
         #["P14", "V:/Data/Analysis/RNLI/participant 14.xdf"],
         #["P15 - 156103", "V:/Data/Analysis/RNLI/participant 15.xdf"],
         #["P16 - 701399", "V:/Data/Analysis/RNLI/participant 16.xdf"]
     ])

In [ ]:
_experiment_analytics: ExperimentAnalytics = ExperimentAnalytics.create_from_paths(participant_data())
_experiment_analytics: ExperimentAnalytics = _experiment_analytics.get_filtered_subset([
    marker_filter(),
    stream_filter()
])

In [ ]:
_session_analytics = _experiment_analytics.analytics_dataframe.loc[_experiment_analytics.analytics_dataframe["Session_ID"] == "P05", "Analysis_Object"].iloc[0]

_session_analytics.localise_timestamps()

_marker_pairs = _session_analytics.get_paired_markers("Marker", "VideoStart", "VideoEnd")

In [ ]:
StreamProcesses.butterworth_filter(_session_analytics, "PPG_GRN", "PPG_GRN_Filtered")
StreamProcesses.butterworth_filter(_session_analytics, "PPG_RED", "PPG_RED_Filtered")
StreamProcesses.butterworth_filter(_session_analytics, "PPG_IR", "PPG_IR_Filtered")

In [ ]:
StreamProcesses.detect_ppg_peaks(_session_analytics, "PPG_GRN_Filtered")
StreamProcesses.detect_ppg_peaks(_session_analytics, "PPG_RED_Filtered")
StreamProcesses.detect_ppg_peaks(_session_analytics, "PPG_IR_Filtered")

In [ ]:
def plot_experiment_data(session_analytics: SessionAnalytics):
    for video_index, row in _marker_pairs.iterrows():
        print(f"[ {video_index + 1} ] {row["Start Marker"]}")
        start_time = row["Start Timestamp"]
        end_time = row["End Timestamp"]

        start_of_calibration = start_time - 60
        duration = end_time - start_of_calibration

        video_analytics = session_analytics.get_filtered_subset([TimestampStreamFilter(start_of_calibration, end_time)])

        filtered_data = video_analytics.stream_data_dictionary["PPG_GRN_Filtered"].copy()
        systolic_peaks = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Systolic_Peaks"].copy()
        diastolic_trough = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Diastolic_Trough"].copy()
        diastolic_peaks = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Diastolic_Peaks"].copy()
        dicrotic_notch = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Dicrotic_Notch"].copy()

        filtered_data = filtered_data - start_of_calibration
        systolic_peaks = systolic_peaks - start_of_calibration
        diastolic_trough = diastolic_trough - start_of_calibration
        diastolic_peaks = diastolic_peaks - start_of_calibration
        dicrotic_notch = dicrotic_notch - start_of_calibration

        plt.figure(figsize=(duration, 4), dpi=300)

        plt.plot(filtered_data["Timestamp"], filtered_data["Value"])
        plt.scatter(systolic_peaks["Timestamp"], systolic_peaks["Value"], c = "orange")
        plt.scatter(diastolic_trough["Timestamp"], diastolic_trough["Value"], c = "blue")
        plt.scatter(diastolic_peaks["Timestamp"], diastolic_peaks["Value"], c = "orange", marker = "x")
        plt.scatter(dicrotic_notch["Timestamp"], dicrotic_notch["Value"], c = "blue", marker = "x")

        plt.vlines(start_time, plt.ylim()[0], plt.ylim()[1], colors="red")
        plt.xlim(0, end_time - start_of_calibration)

        plt.title("Filtered Stream PPG")
        plt.savefig(f"V:/Exports/Video {video_index + 1} Filtered PPG Data.png", dpi=300)
        plt.close()

plot_experiment_data(_session_analytics)


In [ ]:
def process_heartrate(session_analytics: SessionAnalytics):
    for video_index, row in _marker_pairs.iterrows():
        video_start_time = row["Start Timestamp"]
        video_end_time = row["End Timestamp"]

        start_of_previous_calibration = video_start_time - 60
        end_of_next_calibration = video_end_time + 60

        duration = video_end_time - start_of_previous_calibration

        video_analytics = session_analytics.get_filtered_subset([TimestampStreamFilter(start_of_previous_calibration, end_of_next_calibration)])

        systolic_peaks = video_analytics.stream_data_dictionary["PPG_GRN_Filtered_Systolic_Peaks"]


        #region [ Calculate BPM ]
        sample_size: float = 20
        sample_step_count: float = 5

        bpm_dataframe = pd.DataFrame(columns=["Timestamp", "BPM"])

        min_bpm: float = float("inf")
        max_bpm: float = 0

        sample_time = start_of_previous_calibration + (sample_size / 2)
        while sample_time < video_end_time:
            peaks_subset = systolic_peaks[
                (systolic_peaks["Timestamp"] > (sample_time - (sample_size / 2))) &
                (systolic_peaks["Timestamp"] < (sample_time + (sample_size / 2)))
            ]

            bpm = len(peaks_subset) * (60 / sample_size)

            if bpm < min_bpm: min_bpm = bpm
            if bpm > max_bpm: max_bpm = bpm

            bpm_dataframe.loc[len(bpm_dataframe)] = [
                sample_time,
                bpm
            ]

            sample_time = sample_time + sample_step_count
        #endregion

        #region [ Calculate Baseline Heartrate ]
        baseline_sample_subset = bpm_dataframe[
            (bpm_dataframe["Timestamp"] > start_of_previous_calibration + 20) &
            (bpm_dataframe["Timestamp"] < video_start_time)
        ]

        mean_baseline = baseline_sample_subset["BPM"].mean()
        #endregion

        #region [ Plot Data ]
        plt.figure(figsize=(duration * 0.2, 4), dpi=300)
        plt.ylim(40, 120)
        plt.xlim(start_of_previous_calibration, video_end_time)

        # Plot Baseline Heart Rate
        plt.hlines(mean_baseline, start_of_previous_calibration, video_end_time, colors="red", linestyles="dashed", alpha=0.25)

        # Plot Min and Max
        plt.hlines(min_bpm, start_of_previous_calibration, video_end_time, colors="gray", linestyles="dashed", alpha=0.25)
        plt.hlines(max_bpm, start_of_previous_calibration, video_end_time, colors="gray", linestyles="dashed", alpha=0.25)

        #Plot Key Time Points
        plt.vlines(video_start_time, plt.ylim()[0], plt.ylim()[1], colors="cyan", alpha=0.25) # Video Start Time
        plt.axvspan(xmin=start_of_previous_calibration + 20, xmax=video_start_time, color="red", alpha=0.25)

        # Plot BPM Data
        plt.plot(bpm_dataframe["Timestamp"], bpm_dataframe["BPM"])
        #endregion



        plt.title(f"Video {video_index + 1} BPM")
        plt.savefig(f"V:/Exports/Video {video_index + 1} BPM.png", dpi=300)
        plt.close()

process_heartrate(_session_analytics)